<a href="https://colab.research.google.com/github/poojashreeNS/Vertex_AI/blob/main/Vertex_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install additional packages

In [1]:
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

In [2]:
! pip install {USER_FLAG} --upgrade git+https://github.com/googleapis/python-aiplatform.git@v1.6.0

  Cloning https://github.com/googleapis/python-aiplatform.git (to revision v1.6.0) to /tmp/pip-req-build-wt6xdko_
  Running command git clone -q https://github.com/googleapis/python-aiplatform.git /tmp/pip-req-build-wt6xdko_
  Running command git checkout -q 293809edba6bd9275f9be047b4bdd33da17f025f
     |████████████████████████████████| 45 kB 2.2 MB/s 
     |████████████████████████████████| 106 kB 24.1 MB/s 
     |████████████████████████████████| 105 kB 79.0 MB/s 
     |████████████████████████████████| 105 kB 68.2 MB/s 
     |████████████████████████████████| 105 kB 63.2 MB/s 
     |████████████████████████████████| 105 kB 72.0 MB/s 
     |████████████████████████████████| 105 kB 46.2 MB/s 
     |████████████████████████████████| 104 kB 69.5 MB/s 
     |████████████████████████████████| 104 kB 71.7 MB/s 
     |████████████████████████████████| 103 kB 72.9 MB/s 
     |████████████████████████████████| 103 kB 71.6 MB/s 
     |████████████████████████████████| 103 kB 68.5 MB/s 
     |

### Restart the kernel

In [3]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [ ]:
import os

PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Otherwise, set your project ID here.

In [2]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "vertexassignment"  # @param {type:"string"}

In [3]:
print(PROJECT_ID)

vertexassignment


In [4]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# If on Google Cloud Notebooks, then don't execute this code
if not IS_GOOGLE_CLOUD_NOTEBOOK:
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

## Prepare for output

### Step 1. Create dataset for output

We need a BigQuery dataset to host the output data in `us-central1`. Input the name of the dataset you want to created and specify the name of the table you want to store the output later. These will be used later in the notebook.


In [5]:
from datetime import datetime

from google.cloud import bigquery

In [6]:
# Output dataset
DESTINATION_DATA_SET = "movie_predictions"  # @param {type:"string"}
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
DESTINATION_DATA_SET = "{prefix}_{timestamp}".format(
    prefix=DESTINATION_DATA_SET, timestamp=TIMESTAMP
)

# Output table. Make sure that the table does NOT already exist; the BatchReadFeatureValues API cannot overwrite an existing table
DESTINATION_TABLE_NAME = "training_data"  # @param {type:"string"}

DESTINATION_PATTERN = "bq://{project}.{dataset}.{table}"
DESTINATION_TABLE_URI = DESTINATION_PATTERN.format(
    project=PROJECT_ID, dataset=DESTINATION_DATA_SET, table=DESTINATION_TABLE_NAME
)

In [7]:
# Create dataset
REGION = "us-central1"  # @param {type:"string"}
client = bigquery.Client(project=PROJECT_ID)
dataset_id = "{}.{}".format(client.project, DESTINATION_DATA_SET)
dataset = bigquery.Dataset(dataset_id)
dataset.location = REGION
dataset = client.create_dataset(dataset)
print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

Created dataset vertexassignment.movie_predictions_20211211035834


### Import libraries and define constants

In [8]:
# Other than project ID and featurestore ID and endpoints needs to be set
API_ENDPOINT = "us-central1-aiplatform.googleapis.com"  # @param {type:"string"}
INPUT_CSV_FILE = "gs://cloud-samples-data-us-central1/vertex-ai/feature-store/datasets/movie_prediction.csv"

In [9]:
from google.cloud.aiplatform_v1 import (FeaturestoreOnlineServingServiceClient,
                                        FeaturestoreServiceClient)
from google.cloud.aiplatform_v1.types import FeatureSelector, IdMatcher
from google.cloud.aiplatform_v1.types import entity_type as entity_type_pb2
from google.cloud.aiplatform_v1.types import feature as feature_pb2
from google.cloud.aiplatform_v1.types import featurestore as featurestore_pb2
from google.cloud.aiplatform_v1.types import \
    featurestore_online_service as featurestore_online_service_pb2
from google.cloud.aiplatform_v1.types import \
    featurestore_service as featurestore_service_pb2
from google.cloud.aiplatform_v1.types import io as io_pb2
from google.protobuf.duration_pb2 import Duration

# Create admin_client for CRUD and data_client for reading feature values.
admin_client = FeaturestoreServiceClient(client_options={"api_endpoint": API_ENDPOINT})
data_client = FeaturestoreOnlineServingServiceClient(
    client_options={"api_endpoint": API_ENDPOINT}
)

# Represents featurestore resource path.
BASE_RESOURCE_PATH = admin_client.common_location_path(PROJECT_ID, REGION)

### Create Featurestore

The method to create a featurestore returns a
[long-running operation](https://google.aip.dev/151) (LRO). An LRO starts an asynchronous job. LROs are returned for other API
methods too, such as updating or deleting a featurestore. Calling
`create_fs_lro.result()` waits for the LRO to complete.

In [11]:
FEATURESTORE_ID = "movie_prediction"
create_lro = admin_client.create_featurestore(
    featurestore_service_pb2.CreateFeaturestoreRequest(
        parent=BASE_RESOURCE_PATH,
        featurestore_id=FEATURESTORE_ID,
        featurestore=featurestore_pb2.Featurestore(
            online_serving_config=featurestore_pb2.Featurestore.OnlineServingConfig(
                fixed_node_count=1
            ),
        ),
    )
)

In [12]:
# Wait for LRO to finish and get the LRO result.
print(create_lro.result())

name: "projects/451795182630/locations/us-central1/featurestores/movie_prediction"



In [13]:
admin_client.get_featurestore(
    name=admin_client.featurestore_path(PROJECT_ID, REGION, FEATURESTORE_ID)
)

name: "projects/451795182630/locations/us-central1/featurestores/movie_prediction"
create_time {
  seconds: 1639195358
  nanos: 135658000
}
update_time {
  seconds: 1639195358
  nanos: 279282000
}
etag: "AMEw9yORJHl-GH_pXmF5oEQsH_oL0Cnvyzjl0IjwGUmlBPepMKDu7bI_KW5q2zB3DRBj"
online_serving_config {
  fixed_node_count: 1
}
state: STABLE

### Create Entity Type

In [14]:
users_entity_type_lro = admin_client.create_entity_type(
    featurestore_service_pb2.CreateEntityTypeRequest(
        parent=admin_client.featurestore_path(PROJECT_ID, REGION, FEATURESTORE_ID),
        entity_type_id="users",
        entity_type=entity_type_pb2.EntityType(
            description="Users entity",
        ),
    )
)

# Similarly, wait for EntityType creation operation.
print(users_entity_type_lro.result())

name: "projects/451795182630/locations/us-central1/featurestores/movie_prediction/entityTypes/users"



In [15]:
# Create movies entity type without a monitoring configuration.
movies_entity_type_lro = admin_client.create_entity_type(
    featurestore_service_pb2.CreateEntityTypeRequest(
        parent=admin_client.featurestore_path(PROJECT_ID, REGION, FEATURESTORE_ID),
        entity_type_id="movies",
        entity_type=entity_type_pb2.EntityType(description="Movies entity"),
    )
)

# Similarly, wait for EntityType creation operation.
print(movies_entity_type_lro.result())

name: "projects/451795182630/locations/us-central1/featurestores/movie_prediction/entityTypes/movies"



Feature [monitoring](https://cloud.google.com/vertex-ai/docs/featurestore/monitoring) is in preview, we need to use v1beta1 Python. The easiest way to set this for now is using [console UI](https://console.cloud.google.com/vertex-ai/features).


In [16]:
from google.cloud.aiplatform_v1beta1 import \
    FeaturestoreServiceClient as v1beta1_FeaturestoreServiceClient
from google.cloud.aiplatform_v1beta1.types import \
    entity_type as v1beta1_entity_type_pb2
from google.cloud.aiplatform_v1beta1.types import \
    featurestore_monitoring as v1beta1_featurestore_monitoring_pb2
from google.cloud.aiplatform_v1beta1.types import \
    featurestore_service as v1beta1_featurestore_service_pb2

v1beta1_admin_client = v1beta1_FeaturestoreServiceClient(
    client_options={"api_endpoint": API_ENDPOINT}
)

# Enable monitoring for users entity type.
# All Features belonging to this EntityType will by default inherit the monitoring config.
v1beta1_admin_client.update_entity_type(
    v1beta1_featurestore_service_pb2.UpdateEntityTypeRequest(
        entity_type=v1beta1_entity_type_pb2.EntityType(
            name=admin_client.entity_type_path(
                PROJECT_ID, REGION, FEATURESTORE_ID, "users"
            ),
            monitoring_config=v1beta1_featurestore_monitoring_pb2.FeaturestoreMonitoringConfig(
                snapshot_analysis=v1beta1_featurestore_monitoring_pb2.FeaturestoreMonitoringConfig.SnapshotAnalysis(
                    monitoring_interval=Duration(seconds=86400),  # 1 day
                ),
            ),
        ),
    )
)

name: "projects/451795182630/locations/us-central1/featurestores/movie_prediction/entityTypes/users"
description: "Users entity"
create_time {
  seconds: 1639195417
  nanos: 759429000
}
update_time {
  seconds: 1639195429
  nanos: 944245000
}
etag: "AMEw9yO6Vok0XEML4Ku9BvrqRXIIq3uEnBpeaQ5Wc6Y3gxTFAm0k-8SMSgtcUhSIlZqP"
monitoring_config {
  snapshot_analysis {
    monitoring_interval {
      seconds: 86400
    }
    monitoring_interval_days: 1
  }
}

### Create Feature

In [17]:
# Create features for the 'users' entity.
admin_client.batch_create_features(
    parent=admin_client.entity_type_path(PROJECT_ID, REGION, FEATURESTORE_ID, "users"),
    requests=[
        featurestore_service_pb2.CreateFeatureRequest(
            feature=feature_pb2.Feature(
                value_type=feature_pb2.Feature.ValueType.INT64,
                description="User age",
            ),
            feature_id="age",
        ),
        featurestore_service_pb2.CreateFeatureRequest(
            feature=feature_pb2.Feature(
                value_type=feature_pb2.Feature.ValueType.STRING,
                description="User gender",
            ),
            feature_id="gender",
        ),
        featurestore_service_pb2.CreateFeatureRequest(
            feature=feature_pb2.Feature(
                value_type=feature_pb2.Feature.ValueType.STRING_ARRAY,
                description="An array of genres that this user liked",
            ),
            feature_id="liked_genres",
        ),
    ],
).result()

features {
  name: "projects/451795182630/locations/us-central1/featurestores/movie_prediction/entityTypes/users/features/age"
}
features {
  name: "projects/451795182630/locations/us-central1/featurestores/movie_prediction/entityTypes/users/features/gender"
}
features {
  name: "projects/451795182630/locations/us-central1/featurestores/movie_prediction/entityTypes/users/features/liked_genres"
}

In [18]:
# Create features for movies type.
# 'title' Feature enables monitoring.
admin_client.batch_create_features(
    parent=admin_client.entity_type_path(PROJECT_ID, REGION, FEATURESTORE_ID, "movies"),
    requests=[
        featurestore_service_pb2.CreateFeatureRequest(
            feature=feature_pb2.Feature(
                value_type=feature_pb2.Feature.ValueType.STRING,
                description="The title of the movie",
            ),
            feature_id="title",
        ),
        featurestore_service_pb2.CreateFeatureRequest(
            feature=feature_pb2.Feature(
                value_type=feature_pb2.Feature.ValueType.STRING,
                description="The genres of the movie",
            ),
            feature_id="genres",
        ),
        featurestore_service_pb2.CreateFeatureRequest(
            feature=feature_pb2.Feature(
                value_type=feature_pb2.Feature.ValueType.DOUBLE,
                description="The average rating for the movie, range is [1.0-5.0]",
            ),
            feature_id="average_rating",
        ),
    ],
).result()

features {
  name: "projects/451795182630/locations/us-central1/featurestores/movie_prediction/entityTypes/movies/features/title"
}
features {
  name: "projects/451795182630/locations/us-central1/featurestores/movie_prediction/entityTypes/movies/features/genres"
}
features {
  name: "projects/451795182630/locations/us-central1/featurestores/movie_prediction/entityTypes/movies/features/average_rating"
}

## Search created features

In [19]:
# Search for all features across all featurestores.
list(admin_client.search_features(location=BASE_RESOURCE_PATH))

[name: "projects/451795182630/locations/us-central1/featurestores/movie_prediction/entityTypes/movies/features/average_rating"
 description: "The average rating for the movie, range is [1.0-5.0]"
 value_type: DOUBLE
 create_time {
   seconds: 1639195443
   nanos: 595768000
 }
 update_time {
   seconds: 1639195443
   nanos: 595768000
 },
 name: "projects/451795182630/locations/us-central1/featurestores/movie_prediction/entityTypes/movies/features/genres"
 description: "The genres of the movie"
 value_type: STRING
 create_time {
   seconds: 1639195443
   nanos: 594609000
 }
 update_time {
   seconds: 1639195443
   nanos: 594609000
 },
 name: "projects/451795182630/locations/us-central1/featurestores/movie_prediction/entityTypes/movies/features/title"
 description: "The title of the movie"
 value_type: STRING
 create_time {
   seconds: 1639195443
   nanos: 593528000
 }
 update_time {
   seconds: 1639195443
   nanos: 593528000
 },
 name: "projects/451795182630/locations/us-central1/feature

Now, narrow down the search to features that are of type `DOUBLE`

In [20]:
# Search for all features with value type `DOUBLE`
list(
    admin_client.search_features(
        featurestore_service_pb2.SearchFeaturesRequest(
            location=BASE_RESOURCE_PATH, query="value_type=DOUBLE"
        )
    )
)

[name: "projects/451795182630/locations/us-central1/featurestores/movie_prediction/entityTypes/movies/features/average_rating"
 description: "The average rating for the movie, range is [1.0-5.0]"
 value_type: DOUBLE
 create_time {
   seconds: 1639195443
   nanos: 595768000
 }
 update_time {
   seconds: 1639195443
   nanos: 595768000
 }]

Or, limit the search results to features with specific keywords in their ID and type.

In [21]:
# Filter on feature value type and keywords.
list(
    admin_client.search_features(
        featurestore_service_pb2.SearchFeaturesRequest(
            location=BASE_RESOURCE_PATH, query="feature_id:title AND value_type=STRING"
        )
    )
)

[name: "projects/451795182630/locations/us-central1/featurestores/movie_prediction/entityTypes/movies/features/title"
 description: "The title of the movie"
 value_type: STRING
 create_time {
   seconds: 1639195443
   nanos: 593528000
 }
 update_time {
   seconds: 1639195443
   nanos: 593528000
 }]

### Import feature values for Users

When importing, specify the following in your request:

*   Data source format: BigQuery Table/Avro/CSV
*   Data source URL
*   Destination: featurestore/entity types/features to be imported


In [22]:
import_users_request = featurestore_service_pb2.ImportFeatureValuesRequest(
    entity_type=admin_client.entity_type_path(
        PROJECT_ID, REGION, FEATURESTORE_ID, "users"
    ),
    avro_source=io_pb2.AvroSource(
        # Source
        gcs_source=io_pb2.GcsSource(
            uris=[
                "gs://cloud-samples-data-us-central1/vertex-ai/feature-store/datasets/users.avro"
            ]
        )
    ),
    entity_id_field="user_id",
    feature_specs=[
        # Features
        featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(id="age"),
        featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(id="gender"),
        featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(
            id="liked_genres"
        ),
    ],
    feature_time_field="update_time",
    worker_count=1,
)

In [23]:
# Start to import, will take a couple of minutes
ingestion_lro = admin_client.import_feature_values(import_users_request)

In [24]:
# Polls for the LRO status and prints when the LRO has completed
ingestion_lro.result()

imported_entity_count: 7
imported_feature_value_count: 12

### Import feature values for Movies

Similarly, import feature values for 'movies' into the featurestore.


In [25]:
import_movie_request = featurestore_service_pb2.ImportFeatureValuesRequest(
    entity_type=admin_client.entity_type_path(
        PROJECT_ID, REGION, FEATURESTORE_ID, "movies"
    ),
    avro_source=io_pb2.AvroSource(
        gcs_source=io_pb2.GcsSource(
            uris=[
                "gs://cloud-samples-data-us-central1/vertex-ai/feature-store/datasets/movies.avro"
            ]
        )
    ),
    entity_id_field="movie_id",
    feature_specs=[
        featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(id="title"),
        featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(id="genres"),
        featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(
            id="average_rating"
        ),
    ],
    feature_time_field="update_time",
    worker_count=1,
)

In [26]:
# Start to import, will take a couple of minutes
ingestion_lro = admin_client.import_feature_values(import_movie_request)

In [27]:
# Polls for the LRO status and prints when the LRO has completed
ingestion_lro.result()

imported_entity_count: 4
imported_feature_value_count: 12

## Online serving

The
[Online Serving APIs](https://cloud.google.com/vertex-ai/docs/reference/rpc/google.cloud.aiplatform.v1beta1#featurestoreonlineservingservice)
lets you serve feature values for small batches of entities. It's designed for latency-sensitive service, such as online model prediction. For example, for a movie service, we might want to quickly shows movies that the current user would most likely watch by using online predictions.

### Read one entity per request

In [28]:
# Fetch the following 3 features.
feature_selector = FeatureSelector(
    id_matcher=IdMatcher(ids=["age", "gender", "liked_genres"])
)

data_client.read_feature_values(
    featurestore_online_service_pb2.ReadFeatureValuesRequest(
        # Fetch from the following feature store/entity type
        entity_type=admin_client.entity_type_path(
            PROJECT_ID, REGION, FEATURESTORE_ID, "users"
        ),
        # Fetch the user features whose ID is "alice"
        entity_id="alice",
        feature_selector=feature_selector,
    )
)

header {
  entity_type: "projects/451795182630/locations/us-central1/featurestores/movie_prediction/entityTypes/users"
  feature_descriptors {
    id: "age"
  }
  feature_descriptors {
    id: "gender"
  }
  feature_descriptors {
    id: "liked_genres"
  }
}
entity_view {
  entity_id: "alice"
  data {
    value {
      int64_value: 55
      metadata {
        generate_time {
          seconds: 1629493102
          nanos: 261000000
        }
      }
    }
  }
  data {
    value {
      string_value: "Female"
      metadata {
        generate_time {
          seconds: 1629493102
          nanos: 261000000
        }
      }
    }
  }
  data {
    value {
      string_array_value {
        values: "Drama"
      }
      metadata {
        generate_time {
          seconds: 1629493102
          nanos: 261000000
        }
      }
    }
  }
}

### Read multiple entities per request

In [29]:
# Read the same set of features as above, but for multiple entities.
response_stream = data_client.streaming_read_feature_values(
    featurestore_online_service_pb2.StreamingReadFeatureValuesRequest(
        entity_type=admin_client.entity_type_path(
            PROJECT_ID, REGION, FEATURESTORE_ID, "users"
        ),
        entity_ids=["alice", "bob"],
        feature_selector=feature_selector,
    )
)

In [30]:
# Iterate and process response. Note the first one is always the header only.
for response in response_stream:
    print(response)

header {
  entity_type: "projects/451795182630/locations/us-central1/featurestores/movie_prediction/entityTypes/users"
  feature_descriptors {
    id: "age"
  }
  feature_descriptors {
    id: "gender"
  }
  feature_descriptors {
    id: "liked_genres"
  }
}

entity_view {
  entity_id: "alice"
  data {
    value {
      int64_value: 55
      metadata {
        generate_time {
          seconds: 1629493102
          nanos: 261000000
        }
      }
    }
  }
  data {
    value {
      string_value: "Female"
      metadata {
        generate_time {
          seconds: 1629493102
          nanos: 261000000
        }
      }
    }
  }
  data {
    value {
      string_array_value {
        values: "Drama"
      }
      metadata {
        generate_time {
          seconds: 1629493102
          nanos: 261000000
        }
      }
    }
  }
}

entity_view {
  entity_id: "bob"
  data {
    value {
      int64_value: 35
      metadata {
        generate_time {
          seconds: 1629493102
    

### Batch Read Feature Values

Assemble the request which specify the following info:

*   Where is the label data, i.e., Table 1.
*   Which features are read, i.e., the column names in Table 2.

The output is stored in a BigQuery table.


In [31]:
batch_serving_request = featurestore_service_pb2.BatchReadFeatureValuesRequest(
    # featurestore info
    featurestore=admin_client.featurestore_path(PROJECT_ID, REGION, FEATURESTORE_ID),
    # URL for the label data, i.e., Table 1.
    csv_read_instances=io_pb2.CsvSource(
        gcs_source=io_pb2.GcsSource(uris=[INPUT_CSV_FILE])
    ),
    destination=featurestore_service_pb2.FeatureValueDestination(
        bigquery_destination=io_pb2.BigQueryDestination(
            # Output to BigQuery table created earlier
            output_uri=DESTINATION_TABLE_URI
        )
    ),
    entity_type_specs=[
        featurestore_service_pb2.BatchReadFeatureValuesRequest.EntityTypeSpec(
            # Read the 'age', 'gender' and 'liked_genres' features from the 'users' entity
            entity_type_id="users",
            feature_selector=FeatureSelector(
                id_matcher=IdMatcher(
                    ids=[
                        # features, use "*" if you want to select all features within this entity type
                        "age",
                        "gender",
                        "liked_genres",
                    ]
                )
            ),
        ),
        featurestore_service_pb2.BatchReadFeatureValuesRequest.EntityTypeSpec(
            # Read the 'average_rating' and 'genres' feature values of the 'movies' entity
            entity_type_id="movies",
            feature_selector=FeatureSelector(
                id_matcher=IdMatcher(ids=["average_rating", "genres"])
            ),
        ),
    ],
)

In [32]:
# Execute the batch read
batch_serving_lro = admin_client.batch_read_feature_values(batch_serving_request)

In [33]:
# This long runing operation will poll until the batch read finishes.
batch_serving_lro.result()

## Cleaning up

To clean up all Google Cloud resources used in this project, we can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the assignment.

In [ ]:
admin_client.delete_featurestore(
    request=featurestore_service_pb2.DeleteFeaturestoreRequest(
        name=admin_client.featurestore_path(PROJECT_ID, REGION, FEATURESTORE_ID),
        force=True,
    )
).result()
client.delete_dataset(
    DESTINATION_DATA_SET, delete_contents=True, not_found_ok=True
)  # Make an API request.

print("Deleted dataset '{}'.".format(DESTINATION_DATA_SET))